In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reinforcement.players import DummyPlayer, QRModelPlayer
from reinforcement.dqn.dqn_v3 import DQNTrainerV3
from modeling.chessformer import ChessFormerDQN
from torch.optim import AdamW
import torch as t
from copy import deepcopy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BUFFER_SIZE = 1_000_000
RANDOM_ACTION_RATE = 0.0
BATCH_SIZE = 32
NB_STEPS_REWARD = 1
WARM_UP_STEPS = 20_000
ALPHA_SAMPLING = 0.7
BETA_SAMPLING = 0.5
KAPPA=0.001
TAU = 0.0001

In [4]:
model_1 = ChessFormerDQN(6, 3, 512, 2048, 64, 8, 100).to('cuda')
model_2 = ChessFormerDQN(6, 3, 512, 2048, 64, 8, 100).to('cuda')
#model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
#model_1.load_state_dict(model_state_dict)
optimizer = AdamW(model_1.parameters(), lr=1e-4, eps=1e-1)
competitor = DummyPlayer()

In [5]:
trainer = DQNTrainerV3(model_1=model_1,
                       model_2=model_2,
                       optimizer=optimizer,
                       buffer_size=BUFFER_SIZE,
                       competitor=competitor,
                       batch_size=BATCH_SIZE,
                       experiment_name='DQN_v3',
                       model_device='cuda',
                       nb_steps_reward=NB_STEPS_REWARD,
                       warm_up_steps=WARM_UP_STEPS,
                       alpha_sampling=ALPHA_SAMPLING,
                       beta_sampling=BETA_SAMPLING,
                       tau=TAU,
                       kappa=KAPPA)

2023-10-29 11:26:19.705904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 11:26:20.493868: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-29 11:26:20.493936: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-29 11:26:20.493940: W 

In [6]:
trainer.train(15000)

  0%|          | 11/15000 [00:25<10:04:28,  2.42s/it]

In [ ]:
t.save(model_1.state_dict(), 'runs/DQN_v3/model.pt')